In [48]:
# To help with reading and manipulation of data
import numpy as np
import pandas as pd

# To help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To split the data
from sklearn.model_selection import train_test_split

# To impute missing values
from sklearn.impute import SimpleImputer

# To build a Random forest classifier
from sklearn.ensemble import RandomForestClassifier

# To tune a model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# To get different performance metrics
import sklearn.metrics as metrics
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    recall_score,
    accuracy_score,
    precision_score,
    f1_score,
)

# To suppress warnings
import warnings

warnings.filterwarnings("ignore")

# to build logistic regression model
from sklearn.linear_model import LogisticRegression

# to create k folds of data and get cross validation score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# to create pipeline and make_pipeline
from sklearn.pipeline import Pipeline, make_pipeline

# to use standard scaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, StackingRegressor
from sklearn.tree import DecisionTreeClassifier


In [19]:
data = pd.read_csv("Cardiac.csv")
data.head()

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness,UnderRisk
0,Male,1,1,0,1,0,0,0,1,0,0,0,no
1,Male,0,1,0,1,0,0,0,1,0,0,0,no
2,Male,0,1,0,1,0,0,0,1,0,0,0,no
3,Male,0,1,0,1,0,0,0,1,0,0,0,no
4,Male,0,0,0,0,0,1,1,0,0,0,0,no


In [20]:
data["UnderRisk"] = data["UnderRisk"].replace("no", 0)
data["UnderRisk"] = data["UnderRisk"].replace("yes", 1)
data["UnderRisk"].value_counts()

0    699
1    190
Name: UnderRisk, dtype: int64

In [21]:
X = data.drop("UnderRisk", axis=1)
Y = data["UnderRisk"]


In [22]:
X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, train_size=0.8, random_state=1, stratify=Y)
X_train, X_valid,  Y_train, Y_valid = train_test_split(X_temp, Y_temp, test_size=0.25, random_state=1, stratify = Y_temp)


In [23]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_valid = pd.get_dummies(X_valid, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)


In [24]:
X_train["Chain_smoker"].value_counts()

0    471
1     62
Name: Chain_smoker, dtype: int64

In [25]:
sm = SMOTE(sampling_strategy=1, k_neighbors=5, random_state=1)
X_train_over, y_train_over = sm.fit_resample(X_train, Y_train)

In [26]:
rf_tuned=RandomForestRegressor(random_state=1)
rf_tuned.fit(X_train_over,y_train_over)

RandomForestRegressor(random_state=1)

In [27]:
parameters = {  

                'max_depth': list(np.arange(5,30,5)) + [None],

                'max_features': ['sqrt','log2',None],

                'min_samples_leaf': np.arange(1,15,5),

                'min_samples_split': np.arange(2, 20, 5),

                'n_estimators': np.arange(10,110,10)
}
grid_obj = GridSearchCV(rf_tuned, parameters, scoring="recall", cv=5, n_jobs=-1)
grid_obj = grid_obj.fit(X_valid, Y_valid)
grid_obj.best_params_


{'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 10}

In [28]:
#grid_obj.fit(X_valid, Y_valid)


In [44]:
rf2_tuned=RandomForestRegressor(max_depth= 5,
 max_features = 'sqrt',
 min_samples_leaf = 1,
 min_samples_split = 2,
 n_estimators = 10, random_state=1)
rf_tuned.fit(X_train_over,y_train_over)

RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=10,
                      random_state=1)

In [59]:
Ada = AdaBoostRegressor(

n_estimators=70,

learning_rate=0.01,

base_estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
Ada.fit(X_train_over, y_train_over)

TypeError: No loop matching the specified signature and casting was found for ufunc true_divide

In [58]:
Ada.base_estimator.feature_importances_



NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.